# Tutorial: Basic Definitions and Settings

Measuring m/z values is the elemental function of MS technologies, therefore the calculation of mass values for a peptide and its fragments becomes the most essential part in MS-based computational tools. AlphaBase calculates all mass values from atoms. And the masses of amino acids and modifications are calculated from their atom compositions, repectively. Eventually, the masses of peptides or precursors as well as their fragments can be calculated by the amino acid sequences with or without modifications (See figure below).

Calculating masses from atoms makes it much easier to switch between unlabeled and heavy-labeled peptides, as we did in Stellar MS for 15N-labeled peptides as the reference for targeted proteomics (https://www.biorxiv.org/content/10.1101/2024.06.02.597029v2.full).

The other advantage of starting from atoms is that AlphaBase can calculate isotope distributions of peptides based on a pre-defined isotope distribution list of atoms (e.g., NIST atom table in https://physics.nist.gov/cgi-bin/Compositions/stand_alone.pl). The isotope information has been applied in our alphaDIA search engine to boost the identification of DIA-MS data (https://www.biorxiv.org/content/10.1101/2024.05.28.596182v1).

In [ ]:
from IPython.display import Image
Image(filename='atom-to-peptides.png')

### Atoms/Elements

The masses of all amino acids and modifications are calculated from their atom compositions.

The atom information are defined in https://github.com/MannLabs/alphabase/blob/main/alphabase/constants/const_files/nist_element.yaml which is parsed from NIST, see https://github.com/MannLabs/alphabase/blob/main/scripts/nist_chem_to_yaml.ipynb.

After adding some heavy isotopes, including 13C, 15N, 2H, and 18O, we obtain 109 kinds of atoms:

In [ ]:
import pandas as pd
from alphabase.constants.atom import CHEM_INFO_DICT
pd.DataFrame().from_dict(CHEM_INFO_DICT, orient='index')

And their mono-isotopic mass are in `CHEM_MONO_MASS` (dict):

In [ ]:
from alphabase.constants.atom import CHEM_MONO_MASS
pd.DataFrame().from_dict(CHEM_MONO_MASS, orient='index')

These atom masses are used to calculate the masses of amino acids, modifications, and then subsequent masses of peptides and fragments.

#### Commonly used molecular masses

In [ ]:
from alphabase.constants.atom import (
    MASS_PROTON, MASS_ISOTOPE, MASS_NH3, MASS_H2O
)
MASS_PROTON, MASS_ISOTOPE, MASS_NH3, MASS_H2O

### Amino Acids

In [ ]:
from alphabase.constants.aa import AA_DF
AA_DF.loc[ord('A'):ord('Z')]

In `AA_DF`, amino acids are encoded by ASCII (128 characters), thus 65==ord('A'), ..., 90==ord('Z'). Unicode strings can be fastly converted to ascii int32 values using `np.array.view()`:

In [ ]:
import numpy as np

np.array(['ABCXYZ']).view(np.int32)

But users does not need to know this, as we provided easy to use functionalities to get residue masses from sequences.

#### Calculate AA masses in batch

In [ ]:
from alphabase.constants.aa import calc_AA_masses_for_same_len_seqs
calc_AA_masses_for_same_len_seqs(
    [
        'MACDEFG', 'MAKDEFG', 'MAKDEFR'
    ]
)

In [ ]:
from alphabase.constants.aa import calc_AA_masses_for_var_len_seqs
calc_AA_masses_for_var_len_seqs(
    [
        'M', 'MAK', 'MAKDEFR'
    ])

### Modifications

In AlphaBase, we used `mod_name@aa` to represent a modification, the `mod_name` is from UniMod. We also used `mod_name@Protein_N-term`, `mod_name@Any_N-term` and `mod_name@Any_C-term` for terminal modifications, which follow the UniMod terminal name schema.

The default modification TSV is stored in https://github.com/MannLabs/alphabase/blob/main/alphabase/constants/const_files/modification.tsv, which is loaded upon startup of AlphaBase.
Users can add more modifications into the tsv file (only `mod_name` and `composition` columns are required), e.g. by usin the `unimod_to_tsv.ipynb` notebook.

In [ ]:
from alphabase.constants.modification import MOD_DF
MOD_DF

#### Modification sites

In alphabase, we use 0 and -1 to represent modification site of N-term and C-term, respectively. For other modification sites, we use 1 to n.

In [ ]:
from alphabase.constants.modification import calc_modification_mass

# example: add two modifications and print the array of mass modifications
sequence = 'MACDEFG'
mod_names = ['Acetyl@Any_N-term', 'Carbamidomethyl@C']
mod_sites = [0, 3] # 0 for N-term, 3 for the third amino acid
calc_modification_mass(
    nAA=len(sequence),
    mod_names=mod_names,
    mod_sites=mod_sites
)

In [ ]:
# example: add two modifications and print the array of mass modifications
sequence = 'MAKDEFG'
mod_names = ['Acetyl@Any_N-term', 'Oxidation@M']
mod_sites = [0, 1] # 0 for N-term, 1 for the first amino acid
calc_modification_mass(
    nAA=len(sequence),
    mod_names=mod_names,
    mod_sites=mod_sites
)

Multiple modifications at a single site is supported, for example, in the following example, `K3` contains both `GlyGly@K` and `Dimethyl@K`:

In [ ]:
sequence = 'MAKDEFR'
mod_names = ['GlyGly@K', 'Dimethyl@K']
mod_sites = [3, 3]
calc_modification_mass(
    nAA=len(sequence),
    mod_names=mod_names,
    mod_sites=mod_sites
)

#### Caculate modification masses in batch

In [ ]:
from alphabase.constants.modification import calc_mod_masses_for_same_len_seqs
calc_mod_masses_for_same_len_seqs(
    nAA=7,
    mod_names_list=[
        ['Acetyl@Any_N-term', 'Carbamidomethyl@C'],
        ['Acetyl@Any_N-term', 'Oxidation@M'],
        ['GlyGly@K', 'Dimethyl@K'],
    ],
    mod_sites_list=[
        [0, 3],
        [0, 1],
        [3, 3],
    ]
)

### Mass calculation functionalities

#### Calculate AA and modification masses in batch

In [ ]:
from alphabase.constants.aa import calc_AA_masses_for_same_len_seqs
from alphabase.constants.modification import calc_mod_masses_for_same_len_seqs
mod_masses = calc_mod_masses_for_same_len_seqs(
    nAA=7,
    mod_names_list=[
        ['Acetyl@Any_N-term', 'Carbamidomethyl@C'],
        ['Acetyl@Any_N-term', 'Oxidation@M'],
        ['GlyGly@K', 'Dimethyl@K'],
    ],
    mod_sites_list=[
        [0, 3],
        [0, 1],
        [3, 3],
    ]
)
aa_masses = calc_AA_masses_for_same_len_seqs(
    [
        'MACDEFG', 'MAKDEFG', 'MAKDEFR'
    ]
)
mod_masses+aa_masses

#### np.cumsum to get b-ion neutral masses

In [ ]:
import numpy as np
np.cumsum(aa_masses+mod_masses, axis=1)

#### Mass functionalities in 'mass_calc'

The functionalities for peptide and fragment neutral masses have been implement in `alphabase.peptide.mass_calc`:

In [ ]:
from alphabase.peptide.mass_calc import calc_peptide_masses_for_same_len_seqs

peptide_masses = calc_peptide_masses_for_same_len_seqs(
    ['MACDEFG', 'MAKDEFG', 'MAKDEFR'],
    mod_list=[
        'Acetyl@Any_N-term;Carbamidomethyl@C',
        'Acetyl@Any_N-term;Oxidation@M',
        'GlyGly@K;Dimethyl@K',
    ],
)
peptide_masses

In [ ]:
from alphabase.peptide.mass_calc import calc_b_y_and_peptide_masses_for_same_len_seqs
b_masses, y_masses, peptide_masses = calc_b_y_and_peptide_masses_for_same_len_seqs(
    ['MACDEFG', 'MAKDEFG', 'MAKDEFR'],
    mod_list=[
        ['Acetyl@Any_N-term', 'Carbamidomethyl@C'],
        ['Acetyl@Any_N-term', 'Oxidation@M'],
        ['GlyGly@K', 'Dimethyl@K'],
    ],
    site_list=[
        [0, 3],
        [0, 1],
        [3, 3],
    ],
)
peptide_masses

In [ ]:
b_masses

In [ ]:
y_masses

### Isotope distribution

`alphabase.constants.isotope.IsotopeDistribution` will calculate the isotope distribution and the mono-isotopic idx in the distribution for a given atom composition. 

For an atom, the mono-isotopic idx (`mono_idx`) points to the highest abundance isotope, so the value is `round(mass of highest isotope - mass of first isotope)`.

In [ ]:
import pandas as pd
from alphabase.constants.atom import CHEM_INFO_DICT
atom_df = pd.DataFrame().from_dict(CHEM_INFO_DICT, orient='index')
def get_mono(masses_abundances):
    masses, abundances = masses_abundances
    return round(masses[np.argmax(abundances)]-masses[0])
atom_df['mono_idx'] = atom_df[['mass','abundance']].apply(
    get_mono, axis=1
)
atom_df

`mono_idx` of an atom composition refers to the sum of the `mono_idx` of all atoms. In AlphaBase, `alphabase.constants.isotope.IsotopeDistribution` calculate both isotope abundance and `mono_idx`. 

For example, `Fe`'s `mono_idx` is 2 (mass from 53.94 to 55.93), 

In [ ]:
atom_df.loc['Fe']

So `C(1)Fe(1)`'s `mono_idx` is also 2:

In [ ]:
from alphabase.constants.isotope import IsotopeDistribution, parse_formula
iso = IsotopeDistribution()
iso.calc_formula_distribution(
    [('C',1),('Fe',1)]
)

But `13C(1)Fe(1)`'s `mono_idx` should be 3:

In [ ]:
iso.calc_formula_distribution(
    [('13C',1),('Fe',1)]
)

The `mono_idx` of unlabeled atom compositions is always 0, no matter how big the compositions are. This means `mono` isotope is not necessary to be the `highest` isotope peak, especially when the composition get larger. Here are three examples from small composition to large ones, we can see that the highest peaks move from 0 to 2.

In [ ]:
from alphabase.constants.isotope import IsotopeDistribution, parse_formula
iso = IsotopeDistribution()

formula = 'C(50)H(50)O(20)Na(1)'
formula = parse_formula(formula)
dist, mono = iso.calc_formula_distribution(formula)
f"mono={mono}, highest={dist.argmax()}", dist

In [ ]:
formula = 'C(100)H(100)O(20)Na(1)'
formula = parse_formula(formula)
dist, mono = iso.calc_formula_distribution(formula)
f"mono={mono}, highest={dist.argmax()}", dist

In [ ]:
formula = 'C(200)H(200)O(40)Na(1)'
formula = parse_formula(formula)
dist, mono = iso.calc_formula_distribution(formula)
f"mono={mono}, highest={dist.argmax()}", dist